## Important Dependencies with their required versions


In [6]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb


## Imports and Setup

In [7]:
# from langchain_community.llms import HuggingFaceHub
# from langchain_community.chat_models.huggingface import ChatHuggingFace
import os
import sys
from io import StringIO
import json
from Model import InitializeModel,InitializeGptModel
from GenerateUnitTest import InitializeTestChain,queryGpt
from utils.PreprocessUtils import *
from utils.FeedbackUtils import *
from utils.LLMUtilis import *
from utils.FuncUtils import *
from DB import *
from dotenv import load_dotenv
load_dotenv()
# from huggingface_hub import notebook_login
# notebook_login()
import pandas as pd    

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
from CustomThread import CustomThread

def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result

## Initialize Agent

In [8]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     GPTNeoForCausalLM,
# )
# from tokenizers.implementations import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer(
#     "vocab.json",
#     "merges.txt",
# )
load_dotenv()
model_id="google/gemma-7b-it"
max_new_tokens= 6000
if 'Mixtral' in model_id:
    max_new_tokens= 20000

llm, chat_model = InitializeModel(os.environ['HF_TOKEN'], model_id,
                                  max_new_tokens= max_new_tokens)
if 'Mixtral' in model_id:
    GenUnitTestChain= InitializeTestChain(llm)
else:
    GenUnitTestChain= InitializeTestChain(chat_model)

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
db= connect_db()
# llm2,llm3=InitializeGptModel(os.environ['OPENAI_API_KEY'])


WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\DELL\AppData\R

## Generate Unit Tests from Code

In [12]:
total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
ExcelFile="RunningLogs.xlsx"
JSONFile="RunningLogs.json"
# Check if excel file exists
if not os.path.exists(ExcelFile):
  try:
    # Create a new empty DataFrame with no data
    df = pd.DataFrame()
    df.to_excel(ExcelFile, index=False)
    print(f"File {ExcelFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {ExcelFile}: {e}")
    exit()
if not os.path.exists(JSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(JSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {JSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {JSONFile}: {e}")
    exit()
# try:
with open(ExcelFile):
    df = pd.read_excel(ExcelFile,engine='openpyxl')

try:
  with open(JSONFile, 'r') as f:
    data = json.load(f)
except FileNotFoundError:
  print(f"Error: File {JSONFile} not found!")
  exit()
except json.JSONDecodeError:
  print(f"Error: Invalid JSON format in {JSONFile}!")
  exit()

if not isinstance(data, list):
  print(f"Error: Data in {file_path} is not a list. Appending rows requires JSON data as a list of dictionaries!")
  exit()

for i in range(0,1):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    codes_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(codes_of_few_shot, test_cases_few_shot)
    # print("fewShotStr is :",fewShotStr)
    try:
        unittest=GenUnitTestChain.invoke({"description":description,"code":code,"test_cases_of_few_shot":fewShotStr})#,"test_cases_of_few_shot":fewShotStr
    except Exception as e:
        print("ERROR in invoking GenUnitTestChain")
        print(e)
        continue
    ############################################

    ### get the response (unit test) code only without natural language
    unittest_code,isIncompleteResponse = get_code_from_response(unittest['text'],funcDefiniton)
    if(isIncompleteResponse):
        incompleteResponses+=1
    #print("unittest_code is ",unittest_code)
    unittest_code = replaceUnitTestCall(unittest_code)
    # codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    codeTobeRun = get_running_code(code,unittest_code)
    # print("unittest_code is: \n ",unittest_code)
    # print("run_code is \n",codeTobeRun)
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)

    
    if(feedbackparsed==''):
        #in case of OK
        num_successful_examples +=1
        OKCases+=1
        print(f"Test example {i+1} succeeded" )
        num_of_assertions = get_num_assertions(unittest_code)
        print("Number of Ran Tests : " , num_of_assertions) 
        print("Number of Succeeded Test : ", num_of_assertions)
        print("Number of Succeeded Test : ", 0)
        total_test_cases += num_of_assertions

    else:
         num_failed_examples +=1
         total_cases, failed_cases = get_failed_testcases(feedback)
         print(f"Test example {i+1} failed" )
         print("Number of Ran Tests : " , total_cases) 
         print("Number of failed Tests : ", failed_cases)
         print("Number of Succeeded Test : ", total_cases - failed_cases)
         total_test_cases += total_cases
         failed_test_cases+= failed_cases
        

    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    new_row=pd.DataFrame({'Description':description,'Code':code,'GeneratedCode':unittest_code,'Feedback':feedbackparsed},index=[0])
    df = df.append(new_row, ignore_index=True)
    # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
    with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
        # df.to_excel(writer, sheet_name=df.sheet_name.tolist()[0], index=False)  # Assuming only one sheet
        df.to_excel(writer, index=False)  # Assuming only one sheet
    jsondata=df.to_dict(orient='records')
    with open(JSONFile, 'w') as f:
        json.dump(jsondata, f, indent=4)


#dump data into xlsx file
# resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
# #resDF.to_excel("GeneratedFeedback.xlsx",index=False)
# resDF.to_json("GeneratedFeedback.json",orient='records')
# print("OKCases are ",OKCases)
print('\n End of Test Cases \n')
print("Total succeeded examples : " ,num_successful_examples)
print("Total failed examples : ", num_failed_examples)
print("Total  testcases : " ,total_test_cases)
print("Total succeeded testcases : ",total_test_cases- failed_test_cases)
print("Total failed testcases : ",failed_test_cases)
with open('Res.txt', 'w') as f:
    f.write('Total succeeded examples : ' + str(num_successful_examples) + '\n')
    f.write('Total failed examples : ' + str(num_failed_examples) + '\n')
    f.write('Total  testcases : ' + str(total_test_cases) + '\n')
    f.write('Total succeeded testcases : ' + str(total_test_cases- failed_test_cases) + '\n')
    f.write('Total failed testcases : ' + str(failed_test_cases) + '\n')

print("Incomplete Responses are ",incompleteResponses)

Running Test Case  1
Test example 1 succeeded
Number of Ran Tests :  6
Number of Succeeded Test :  6
Number of Succeeded Test :  0

 End of Test Cases 

Total succeeded examples :  1
Total failed examples :  0
Total  testcases :  6
Total succeeded testcases :  6
Total failed testcases :  0
Incomplete Responses are  0


C:\Users\DELL\AppData\Local\Temp\ipykernel_19324\2978897245.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [4]:
import pandas as pd
import json
import os
# Define the file path and data for the new row
file_path = "your_file.json"
new_data = {"key1": "value1", "key2": "value2"}  # Replace with your actual keys and values


# Check if the file exists
if not os.path.exists(file_path):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(file_path, 'w') as f:
      json.dump([], f)
    print(f"File {file_path} created successfully!")
  except Exception as e:
    print(f"Error creating file {file_path}: {e}")
    exit()
# Read the existing JSON data (assuming it's a list of dictionaries)
try:
  with open(file_path, 'r') as f:
    data = json.load(f)
except FileNotFoundError:
  print(f"Error: File {file_path} not found!")
  exit()
except json.JSONDecodeError:
  print(f"Error: Invalid JSON format in {file_path}!")
  exit()

# Check if data is a list (expected format for appending rows)
if not isinstance(data, list):
  print(f"Error: Data in {file_path} is not a list. Appending rows requires JSON data as a list of dictionaries!")
  exit()

# Create a DataFrame from the existing data
df = pd.DataFrame(data)

# Create a DataFrame from the new row data
new_row = pd.DataFrame([new_data])

# Append the new row to the existing DataFrame
df = df.append(new_row, ignore_index=True)

# Convert the updated DataFrame back to a list of dictionaries
data = df.to_dict(orient='records')

# Write the updated data back to the JSON file
with open(file_path, 'w') as f:
  json.dump(data, f, indent=4)  # Use indent for readability (optional)

print(f"New row appended to {file_path} successfully!")

New row appended to your_file.json successfully!


C:\Users\DELL\AppData\Local\Temp\ipykernel_19324\1510761095.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


## GPT MODEL

In [ ]:
introCode = """
import sys
sys.setrecursionlimit(100)
"""
import time
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]

for i in range(0,10):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    code_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(code_of_few_shot, test_cases_few_shot)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model
    #try gpt model
    unittest=queryGpt(llm2,description,code,fewshots=False,test_cases_of_few_shot=fewShotStr)['choices'][0]['message']['content']
    print("gpt_response is ",unittest)

    
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model

    ### get the response (unit test) code only without natural language
    unittest_code = get_code_from_response(unittest,funcDefiniton)
    unittest_code = replaceUnitTestCall(unittest_code)
    ##############run code with the original and unit test concatenated #######################
    codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    print("run_code is")
    print(codeTobeRun)
    # print("------------------------------lolooloolololo ------------------------------------")
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    if(feedbackparsed==''):
        #in case of OK
        OKCases+=1
        # feedbackparsed=""
        # for i,line in enumerate(feedback.split('\n')):
        #     if i>1:
        #         feedbackparsed+=line+'\n'
    #print(feedbackparsed)
    print(feedback)
    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    time.sleep(20)

#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
#resDF.to_excel("GeneratedFeedbackGPT.xlsx",index=False)
resDF.to_json("GeneratedFeedbackGPT.json",orient='records')
print("OKCases are ",OKCases)
print("Total Cases are ",len(descriptions))
with open('ResGPT.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")


## Bug Fixing

In [ ]:
BugRepair_template_str = """Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
{code}```

The following are the bugs and test cases corresponding to the bugs:
```python
{TestcaseCode}```

The result after running these test cases on an interpreter:
{InterpreterFeedback}

Change either the code or the test cases according to the feedback of the interpreter.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
"""
InterpreterFeedback=feedbackparsed

BugRepair_Template = PromptTemplate(
    template=BugRepair_template_str,
    input_variables=["code","TestcaseCode","InterpreterFeedback"],
    verbose=True
    )

BugFixChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=BugRepair_Template
)

res=BugFixChain.invoke({
                             "code":code,
                             "TestcaseCode":res_code,
                             "InterpreterFeedback":InterpreterFeedback
                             })
print(res['text'])

## input code preprocessor

In [ ]:
#if the input code is without description, we want to generate description
Generate_description_Template = """Act as a python expert, Describe the purpose and functionality of the following Python code.

```python
{code}```
The output should be like that:
The provided Python code aims to [explain purpose].
In summary, [concise summary of functionality ].
Examples: [3 or 4 input output examples]
"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
Generate_description_PromptTemplate = PromptTemplate(
    template=Generate_description_Template,
    input_variables=["code"],
    verbose=True
    )

GenDescriptionChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_description_PromptTemplate
)
res=GenDescriptionChain.invoke({
                             "code":code
                             })
print(res['text'])